# **과제2 : 챗봇 만들기2**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기2
    * Vector DB에 데이터 추가하기
    * Retriever, memory, LLM를 연결하기
    * 실행시 이력 DB 생성하고 기록하기
    * test

## **1.환경준비**

### (1) 라이브러리 Import

In [2]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

sk-proj-r3rAEj45bQDpougunpTQT3BlbkFJcrlY0aisAKiA0diHhutD


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [3]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 csv 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

In [48]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

In [53]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 웹 페이지 요청
url = 'https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056'
driver.get(url)

# 페이지가 로드될 때까지 대기
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "div#tab_menu"))
)

# 각 탭을 클릭하고 내용 크롤링
tab_menu = driver.find_element(By.ID, 'tab_menu')
tab_links = tab_menu.find_elements(By.CLASS_NAME, 'tab-link')

faq_data = []

for tab in tab_links[1:]:
    tab.click()
    time.sleep(2)  # 페이지가 로드될 시간을 충분히 주기 위해 잠시 대기

    # 현재 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup을 사용하여 HTML 파싱
    html = BeautifulSoup(page_source, 'html.parser')
    
    # 각 탭의 FAQ 데이터 추출
    tab_content_div = html.select_one('div.tabs_content')
    ul_element = tab_content_div.find('ul', class_='toggle_list')
    
    if ul_element:
        li_elements = ul_element.find_all('li')
        
        for li in li_elements:
            faq_data.append(li.get_text(strip=True))

# Selenium 드라이버 종료
driver.quit()

# 크롤링한 데이터 출력
for item in faq_data:
    print(item)

모집/선발1) 최종 학력 또는 전공과 관계없이 지원할 수 있나요?KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.
모집/선발2) 35세 이상은 지원할 수 없나요?본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)
모집/선발3) 미취업자의 기준이 뭔가요?미취업자의 기준은 아래와 같습니다.1) 기간의 정함이 있는 근로인 경우,2) 고용보험에 미가입한 경우,3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.
모집/선발4) 직장인도 지원할 수 있나요?KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다.
향후 취업여부를 확인할 예정입니다.
모집/선발5) 아르바이트를 하고 있는데 지원할 수 있나요?고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다.고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지급되지 않습니다.다만, 특수형태근로자의 경우 해촉증명서, 퇴사 사실확인증명원(사업주, 근로자 직인/서명포함)을 제출하는 경우 훈련장려금 지급이 가능하므로,고용형태 관련 세부사항은 거주지 관할 고용센터(HRD-Net 확인) 또는 훈련 권역별 고용센터로 문의하시기 바랍니다.※ 수도권: 성남고용센터충남/충북: 대전고용센터대구/경북: 대구고용센터전남/전북: 광주고용센터부산/경남: 부산고용센터- 고용노동부 대표번호 1350, 거주지 근처 지도검색, HRD-Net 메

In [67]:
# to pandas dataframe split category and question
df = pd.DataFrame()

categories = []
questions = []
for item in faq_data:
    split_idx = item.find(') ')
    category, question = item[:split_idx], item[split_idx+2:]
    
    # remove category number
    for i in range(-1, -len(category), -1):
        if not category[i].isdigit():
            category = category[:i+1]
            break
    
    categories.append(category)
    questions.append(question)

df = pd.DataFrame({'구분': categories, 'QA': questions})
df.head()


,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?KT 에이블스쿨은 정규 4년제 대...
1,모집/선발,35세 이상은 지원할 수 없나요?본 교육 과정은 34세 이하를 대상으로 하는 교육입...
2,모집/선발,미취업자의 기준이 뭔가요?미취업자의 기준은 아래와 같습니다.1) 기간의 정함이 있는...
3,모집/선발,"직장인도 지원할 수 있나요?KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?고용보험에 가입이 되어 있는 경우 15시...


In [70]:
# save csv
df.to_csv('aivleschool_qa_crawling.csv', index=False, encoding='utf-8-sig')

In [5]:
data = pd.read_csv('aivleschool_qa_2.csv', encoding='utf-8')
data.tail()

,구분,QA
37,기타,AICE (기존 AIFB) 자격증이 있어야 유리한가요?\n\n네. AICE (기존 ...
38,기타,AICE 시험관련 문의는 어디에 해야하나요?\n\nhelp@aice.study 으로...
39,기타,국민취업지원제도를 신청하고 싶습니다.\n\n국민취업지원제도 신청과 관련 사항은 거주...
40,기타,국민취업지원제도를 안해도 교육 참여되나요?\n\n국민취업제도를 신청하지 않아도 KT...
41,기타,국민취업지원제도 관련 직업훈련탐색표가 필요합니다.\n\n직업훈련탐색표는 고용센터 또...


* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./db



In [7]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./db",
                    embedding_function = embeddings
)

* 데이터 입력
    * 기존 입력을 모두 제거하고 추가 사항만 모두 입력
    * meta data로 '구분' 칼럼 값 추가하기

In [18]:
rm_ids = database.get()['ids']
database.delete(rm_ids)
database.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [10]:
data.columns

Index(['구분', 'QA'], dtype='object')

In [20]:
input_list = []
metadata = []
for category, QA_chunk in zip(data['구분'], data['QA'].tolist()):
    input_list.append(QA_chunk)
    metadata.append({"category": category})

* 입력된 데이터 조회

In [21]:
doc = [Document(page_content=input_list[i], metadata=metadata[i]) for i in range(len(input_list))]
ind = database.add_documents(doc)



In [22]:
database.get()

{'ids': ['01bf162c-a83c-4786-b91e-33961a38ed27',
  '02b56f72-6346-4666-970b-910f8787fb63',
  '15f862c3-e036-463c-896b-db04e2506dc4',
  '162eba83-1914-42a0-90a3-c4639c9751c5',
  '18809b61-50c3-4df7-8c74-ce5a77b5d440',
  '203bcb0f-2f1d-4f8d-823c-56acf072be97',
  '2827c5ea-40c0-4fa1-97e7-7c9c4af43ad5',
  '28beefce-2713-4446-875e-65a9c5fcec73',
  '2ef087d3-9865-4a3b-850f-57c3531d2e26',
  '2f63968c-6c1e-4a1b-9943-ce2231bc8358',
  '3596d92e-84c9-4264-a366-bc4819fa2fa3',
  '3c0e9ed7-86ca-4f4c-bac9-ddf11656c234',
  '3d69ce8d-554e-40ed-98df-5d17dfe9a747',
  '40245fa7-36c9-4a0e-83ca-0d940671743f',
  '4e14cd25-dd8b-487d-987e-01e81d9e80a1',
  '5310be6a-798c-435b-a38c-6c65b8d5768d',
  '61549ed3-0238-4d06-bf53-f0bfd64daa08',
  '6b103b3a-bc83-4bab-8b6a-de65f6b16cca',
  '7545f6a7-ac07-4032-9586-cfb6cad0c37e',
  '76281437-3c1f-45bf-b09f-50d210ac1f90',
  '85e26226-351e-40e1-9213-4e9c05337eb0',
  '863428f8-34d6-42b5-8146-810b6aad7232',
  '88261ac7-17c4-42a4-9b55-2d2ed81276d9',
  '8aea1198-387a-422e-8f3a-

* 모델 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정
    * memory 사용
* 요구사항
    * 질문 history 관리를 위한 이력 저장 DB 생성
        * DB 명 : db_chatlog
        * 테이블 명 : history
            * id INTEGER PRIMARY KEY : 이렇게 설정하면 자동증가 값으로 채워짐
            * datetime TEXT : 질문시점 yyyy-mm-dd hh:mi:ss
            * query TEXT : 질문
            * sim1 REAL : 첫번째 문서의 유사도 점수
            * sim2 REAL : 두번째 문서의 유사도 점수
            * sim3 REAL : 세번째 문서의 유사도 점수
            * answer TEXT : 답변
        * 유사도 점수는 similarity_search_with_score 메서드를 이용해서 저장해야 함
        * 질문과 답변이 진행될 때마다 history 테이블에 데이터 입력

## **3.RAG+LLM모델**

* 관리용 DB, 테이블 생성

In [34]:
path = './db/db_chatlog.db'
conn = sqlite3.connect(path)

# 커서 객체 생성
cursor = conn.cursor()

# test 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS history (
    id INTEGER PRIMARY KEY,
    datetime TEXT NOT NULL,
    query TEXT NOT NULL,
    sim1 REAL NOT NULL,
    sim2 REAL NOT NULL,
    sim3 REAL NOT NULL,
    answer TEXT NOT NULL
)
''')

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

* 모델 선언

In [35]:
k = 3
retriever = database.as_retriever(search_kwargs={"k": k})
chat = ChatOpenAI(model="gpt-3.5-turbo")
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")

* 모델 사용 및 이력 확인

In [36]:
conn = sqlite3.connect(path)

while True:
    query = input("질문을 입력하세요: ")
    if query == "":
        break
    dt = datetime.now()
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    scores = database.similarity_search_with_score(query, k=k)
    
    output = qa(query)
    
    data = pd.DataFrame({'datetime': [dt],
                     'query': [query],
                     'sim1': [scores[0][1]],
                     'sim2': [scores[1][1]],
                     'sim3': [scores[2][1]],
                     'answer': [output['answer']]
                     })
    
    data.to_sql('history', conn, if_exists='append', index=False)

conn.close()

In [37]:
conn = sqlite3.connect(path)
df = pd.read_sql('SELECT * FROM history', conn)
conn.close()

display(df)

,id,datetime,query,sim1,sim2,sim3,answer
0,1,2024-06-04 12:11:29,생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?,0.335571,0.345751,0.359818,생성형 AI 도입시 예상되는 보안 위협은 다양할 수 있습니다. 몇 가지 예시로는 데...
1,2,2024-06-04 12:11:45,지원하는데 나이 제한이 있나?,0.271411,0.314145,0.314541,"네, 해당 교육 과정은 34세 이하의 지원자를 대상으로 하고 있습니다. 다만, 모집..."


#################################################################

In [24]:
# 첫번째 질문
query1 = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query1)
result['answer']

'생성형 AI 도입시 예상되는 보안 위협은 다양할 수 있습니다. 몇 가지 가능한 위협은 다음과 같습니다:\n1. 악용된 모델: 악의적인 공격자가 모델을 조작하여 부적절한 결과를 얻거나, 모델을 사용자의 개인 정보를 탈취하는 용도로 악용할 수 있습니다.\n2. 데이터 위조: 모델을 속이기 위해 데이터를 조작하는 것으로, 모델의 성능을 저하시킬 수 있습니다.\n3. 개인 정보 보호: 모델이 개인 정보를 처리하는 경우, 해당 정보가 노출되거나 유출될 수 있습니다.\n4. 편향성: 모델이 편향된 데이터로 학습되면 공정하지 않은 결과를 도출할 수 있습니다.\n이 외에도 다양한 보안 위협이 있을 수 있으며, 보안 전문가와 함께 상세한 위험 분석을 수행하는 것이 중요합니다.'

In [32]:
dt = datetime.now()
dt = dt.strftime('%Y-%m-%d %H:%M:%S')
scores = database.similarity_search_with_score(query1, k=k)
data = pd.DataFrame({'datetime': [dt],
                     'query': [query1],
                     'sim1': [scores[0][1]],
                     'sim2': [scores[1][1]],
                     'sim3': [scores[2][1]],
                     'answer': [result['answer']]
                     })
data.to_sql('history', conn, if_exists='append', index=False)


[(Document(page_content='KT 채용 연계 시 중점적으로 보는 요소가 있나요?\n\nKT 그룹, AI 원팀 기업/스타트업 100 등 다양한 기업에서 채용 연계할 예정으로 기업마다 인재상 및 반영 요소가 다르기 때문에 중점 요소에 대해 안내드리기 어려운 점 양해 부탁드립니다. 채용에 대한 부분은 교육을 참여하시면 추후 관련 부서를 통해 안내드릴 예정입니다.', metadata={'category': '채용연계'}),
  0.33557128629375615),
 (Document(page_content='컴퓨터공학 비전공자도 커리큘럼을 따라갈 수 있을까요?\n\n비전공자도 SW개발에 관심과 열정이 있으신 분들은 충분히 따라 갈 수 있습니다. 이 경우 데이터 사이언스 기반으로 산업 현장에 AI를 접목하고 DX를 이끄는 DX컨설턴트 Track을 추천 드립니다. SW기초 지식 및 코딩역량을 보유하고 있고 개발자로 성장하고 싶은 분께는 AI개발자 Track을 추천 드립니다.  ', metadata={'category': '교육/수강'}),
  0.34575127723303706),
 (Document(page_content='교육 전에 미리 공부하면 좋은 것이 있을까요?\n\n파이썬 프로그래밍 사용법과 AI 개념에 대한 기초 이해를 공부하고 오시면 도움이 되실 것 같습니다.', metadata={'category': '교육/수강'}),
  0.3598177318099352)]

In [25]:
result

{'question': '생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?',
 'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 도입시 예상되는 보안 위협은 다양할 수 있습니다. 몇 가지 가능한 위협은 다음과 같습니다:\n1. 악용된 모델: 악의적인 공격자가 모델을 조작하여 부적절한 결과를 얻거나, 모델을 사용자의 개인 정보를 탈취하는 용도로 악용할 수 있습니다.\n2. 데이터 위조: 모델을 속이기 위해 데이터를 조작하는 것으로, 모델의 성능을 저하시킬 수 있습니다.\n3. 개인 정보 보호: 모델이 개인 정보를 처리하는 경우, 해당 정보가 노출되거나 유출될 수 있습니다.\n4. 편향성: 모델이 편향된 데이터로 학습되면 공정하지 않은 결과를 도출할 수 있습니다.\n이 외에도 다양한 보안 위협이 있을 수 있으며, 보안 전문가와 함께 상세한 위험 분석을 수행하는 것이 중요합니다.')],
 'answer': '생성형 AI 도입시 예상되는 보안 위협은 다양할 수 있습니다. 몇 가지 가능한 위협은 다음과 같습니다:\n1. 악용된 모델: 악의적인 공격자가 모델을 조작하여 부적절한 결과를 얻거나, 모델을 사용자의 개인 정보를 탈취하는 용도로 악용할 수 있습니다.\n2. 데이터 위조: 모델을 속이기 위해 데이터를 조작하는 것으로, 모델의 성능을 저하시킬 수 있습니다.\n3. 개인 정보 보호: 모델이 개인 정보를 처리하는 경우, 해당 정보가 노출되거나 유출될 수 있습니다.\n4. 편향성: 모델이 편향된 데이터로 학습되면 공정하지 않은 결과를 도출할 수 있습니다.\n이 외에도 다양한 보안 위협이 있을 수 있으며, 보안 전문가와 함께 상세한 위험 분석을 수행하는 것이 중요합니다.',
 'source_documents': [Document(page_content='KT 채용 연계 시 중점적으로 보는 요소가 있나요?\n\nKT 그룹, AI

In [26]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 도입시 예상되는 보안 위협은 다양할 수 있습니다. 몇 가지 가능한 위협은 다음과 같습니다:\n1. 악용된 모델: 악의적인 공격자가 모델을 조작하여 부적절한 결과를 얻거나, 모델을 사용자의 개인 정보를 탈취하는 용도로 악용할 수 있습니다.\n2. 데이터 위조: 모델을 속이기 위해 데이터를 조작하는 것으로, 모델의 성능을 저하시킬 수 있습니다.\n3. 개인 정보 보호: 모델이 개인 정보를 처리하는 경우, 해당 정보가 노출되거나 유출될 수 있습니다.\n4. 편향성: 모델이 편향된 데이터로 학습되면 공정하지 않은 결과를 도출할 수 있습니다.\n이 외에도 다양한 보안 위협이 있을 수 있으며, 보안 전문가와 함께 상세한 위험 분석을 수행하는 것이 중요합니다.')]}

In [27]:
# from langchain.schema import HumanMessage, SystemMessage, chat_history
